# MindQuantum全面介绍

In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

In [2]:
from mindquantum import *
import numpy as np

## 量子门

In [3]:
H

H

In [4]:
H.on(0)

H(0)

In [5]:
H.matrix()

array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])

In [6]:
X.on(1,0)

X(1 <-: 0)

In [7]:
RX(np.pi)

RX(π)

In [8]:
RX(np.pi).matrix()

array([[6.123234e-17+0.j, 0.000000e+00-1.j],
       [0.000000e+00-1.j, 6.123234e-17+0.j]])

In [9]:
RX('a')

RX(a)

In [10]:
RX('a').matrix({'a':np.pi})

array([[6.123234e-17+0.j, 0.000000e+00-1.j],
       [0.000000e+00-1.j, 6.123234e-17+0.j]])

## 量子线路

In [11]:
circuit = Circuit()

In [12]:
circuit = Circuit([H.on(0), X.on(1, 0)])
circuit

q0: ──H────●──
           │  
q1: ───────X──

In [13]:
circuit += qft(range(3))
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@──
           │            │          │                            │  
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼──
                                   │               │            │  
q2: ───────────────────────────────●───────────────●───────H────@──

In [14]:
circuit += ZZ('alpha').on([0, 1])
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@────ZZ(alpha)──
           │            │          │                            │        │      
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼────ZZ(alpha)──
                                   │               │            │               
q2: ───────────────────────────────●───────────────●───────H────@───────────────

In [15]:
circuit += BARRIER
circuit += H.on(2)

In [16]:
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@────ZZ(alpha)───────
           │            │          │                            │        │           
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼────ZZ(alpha)───────
                                   │               │            │                    
q2: ───────────────────────────────●───────────────●───────H────@─────────────────H──

In [17]:
circuit += H.on(0)
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@────ZZ(alpha)────H──
           │            │          │                            │        │           
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼────ZZ(alpha)───────
                                   │               │            │                    
q2: ───────────────────────────────●───────────────●───────H────@─────────────────H──

In [18]:
circuit += BarrierGate(show=False)
circuit += Measure('q1').on(1)
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@────ZZ(alpha)────H───────────
           │            │          │                            │        │                    
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼────ZZ(alpha)─────────M(q1)──
                                   │               │            │                             
q2: ───────────────────────────────●───────────────●───────H────@─────────────────H───────────

## Simulator

In [19]:
sim = Simulator('projectq', 3)
sim

projectq simulator with 3 qubits.
Current quantum state:
1¦000⟩

In [20]:
print(sim.get_qs(True))

1¦000⟩


In [21]:
sim.apply_circuit(qft(range(3)))
sim

projectq simulator with 3 qubits.
Current quantum state:
√2/4¦000⟩
√2/4¦001⟩
√2/4¦010⟩
√2/4¦011⟩
√2/4¦100⟩
√2/4¦101⟩
√2/4¦110⟩
√2/4¦111⟩

In [22]:
sim.reset()
sim

projectq simulator with 3 qubits.
Current quantum state:
1¦000⟩

In [23]:
circ = Circuit().h(0).rx('a', 0).x(1,0).rx('b', 0)
circ.barrier(False).measure_all()
circ += Measure('q1_1').on(0)
circ

q0: ──H────RX(a)────●────RX(b)────M(q0)────M(q1_1)──
                    │                               
q1: ────────────────X─────────────M(q1)─────────────

In [24]:
sim = Simulator('projectq', circ.n_qubits)
res = sim.sampling(circ, {'a': np.pi, 'b': np.pi}, shots=100, seed=42)
res

shots: 100
Keys: q1_1 q1 q0│0.00    0.14        0.28        0.42        0.56         0.7
────────────────┼───────────┴───────────┴───────────┴───────────┴───────────┴
             010│▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
                │                                                            
             101│▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
                │                                                            
{'010': 44, '101': 56}

In [25]:
sim.apply_circuit(circ[:4], {'a': np.pi, 'b': np.pi})
print(sim.get_qs(ket=True))

-√2/2¦01⟩
-√2/2¦10⟩


In [26]:
print(circ[:4].get_qs(ket=True, pr={'a':np.pi, 'b':np.pi}))

-√2/2¦01⟩
-√2/2¦10⟩


In [27]:
qft(range(3)).get_qs('projectq')

array([0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
       0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j])

## 参数化泡利算符&费米算符

In [28]:
from mindquantum.core import QubitOperator, FermionOperator
from mindquantum.algorithm.nisq.chem import Transform

In [29]:
fo = FermionOperator('0 1^ 2 3^', 5)
fo

5 [0 1^ 2 3^] 

In [30]:
fo = FermionOperator('0 1^ 2 3^', 'x')
fo

x [0 1^ 2 3^] 

In [31]:
qo = 'y'*QubitOperator('X0 Y1')
qo

1.0*y [X0 Y1] 

In [32]:
f2q = Transform(fo).jordan_wigner()
f2q

0.0625*x [X0 X1 X2 X3] +
-0.0625*I*x [X0 X1 X2 Y3] +
0.0625*I*x [X0 X1 Y2 X3] +
0.0625*x [X0 X1 Y2 Y3] +
-0.0625*I*x [X0 Y1 X2 X3] +
-0.0625*x [X0 Y1 X2 Y3] +
0.0625*x [X0 Y1 Y2 X3] +
-0.0625*I*x [X0 Y1 Y2 Y3] +
0.0625*I*x [Y0 X1 X2 X3] +
0.0625*x [Y0 X1 X2 Y3] +
-0.0625*x [Y0 X1 Y2 X3] +
0.0625*I*x [Y0 X1 Y2 Y3] +
0.0625*x [Y0 Y1 X2 X3] +
-0.0625*I*x [Y0 Y1 X2 Y3] +
0.0625*I*x [Y0 Y1 Y2 X3] +
0.0625*x [Y0 Y1 Y2 Y3] 

In [33]:
f2q_imag = f2q.imag
f2q_imag.compress()

-0.0625*x [X0 X1 X2 Y3] +
0.0625*x [X0 X1 Y2 X3] +
-0.0625*x [X0 Y1 X2 X3] +
-0.0625*x [X0 Y1 Y2 Y3] +
0.0625*x [Y0 X1 X2 X3] +
0.0625*x [Y0 X1 Y2 Y3] +
-0.0625*x [Y0 Y1 X2 Y3] +
0.0625*x [Y0 Y1 Y2 X3] 

In [34]:
ham = f2q_imag + qo
ham 

-0.0625*x [X0 X1 X2 Y3] +
0.0625*x [X0 X1 Y2 X3] +
1.0*y [X0 Y1] +
-0.0625*x [X0 Y1 X2 X3] +
-0.0625*x [X0 Y1 Y2 Y3] +
0.0625*x [Y0 X1 X2 X3] +
0.0625*x [Y0 X1 Y2 Y3] +
-0.0625*x [Y0 Y1 X2 Y3] +
0.0625*x [Y0 Y1 Y2 X3] 

## Trotter分解

In [35]:
from mindquantum.core import TimeEvolution

In [36]:
trotter_circ = TimeEvolution(ham).circuit
trotter_circ

q0: ──RX(π/2)────●───────────────────────────────────────●────RX(7π/2)────RX(π/2)────●───────────────────────────────────────●────RX(7π/2)────RX(π/2)────●───────────────────────────────────────●────RX(7π/2)────RX(π/2)────●────────────────────────────────────────●────RX(7π/2)───────H───────●───────────────────────────────────────●───────H───────────H───────●────────────────────────────────────────●───────H───────────H───────●────────────────────────────────────────●───────H───────────H───────●────────────────────────────────────────●───────H───────────H───────●─────────────────●───────H──────
                 │                                       │                           │                                       │                           │                                       │                           │                                        │                           │                                       │                           │                                        │                           │                                        │                           │                                        │                           │                 │              
q1: ─────H───────X────●─────────────────────────────●────X───────H───────────H───────X────●─────────────────────────────●────X───────H────────RX(π/2)────X────●─────────────────────────────●────X────RX(7π/2)────RX(π/2)────X────●──────────────────────────────●────X────RX(7π/2)───────H───────X────●─────────────────────────────●────X───────H───────────H───────X────●──────────────────────────────●────X───────H────────RX(π/2)────X────●──────────────────────────────●────X────RX(7π/2)────RX(π/2)────X────●──────────────────────────────●────X────RX(7π/2)────RX(π/2)────X────RZ(2.0*y)────X────RX(7π/2)──
                      │                             │                                     │                             │                                     │                             │                                     │                              │                                     │                             │                                     │                              │                                     │                              │                                     │                              │                                                                 
q2: ──RX(π/2)─────────X────●───────────────────●────X─────────RX(7π/2)───────H────────────X────●───────────────────●────X────────────H────────RX(π/2)─────────X────●───────────────────●────X─────────RX(7π/2)───────H────────────X────●────────────────────●────X────────────H────────RX(π/2)─────────X────●───────────────────●────X─────────RX(7π/2)───────H────────────X────●────────────────────●────X────────────H────────RX(π/2)─────────X────●────────────────────●────X─────────RX(7π/2)───────H────────────X────●────────────────────●────X────────────H────────────────────────────────────────────────────
                           │                   │                                               │                   │                                               │                   │                                               │                    │                                               │                   │                                               │                    │                                               │                    │                                               │                    │                                                                      
q3: ──RX(π/2)──────────────X────RZ(0.125*x)────X──────────────RX(7π/2)───────H─────────────────X────RZ(0.125*x)────X─────────────────H───────────H─────────────────X────RZ(0.125*x)────X─────────────────H────────RX(π/2)──────────────X────RZ(-0.125*x)────X──────────────RX(7π/2)───────H─────────────────X────RZ(0.125*x)────X─────────────────H────────RX(π/2)──────────────X────RZ(-0.125*x)────X──────────────RX

In [37]:
ham = ham.subs({'x':1.1, 'y':2.2})
ham

-0.06875 [X0 X1 X2 Y3] +
0.06875 [X0 X1 Y2 X3] +
2.2 [X0 Y1] +
-0.06875 [X0 Y1 X2 X3] +
-0.06875 [X0 Y1 Y2 Y3] +
0.06875 [Y0 X1 X2 X3] +
0.06875 [Y0 X1 Y2 Y3] +
-0.06875 [Y0 Y1 X2 Y3] +
0.06875 [Y0 Y1 Y2 X3] 

In [38]:
from mindquantum.core import commutator
ham2 = commutator(ham, QubitOperator('X0 Y1 Z2 X3'))
ham2 = ham2.imag
ham2

-0.1375 [Z0 Z1 Y2] +
-0.1375 [Z0 Y2 Z3] +
0.1375 [Z1 Y2 Z3] +
0.1375 [Y2] 

In [39]:
ham3 = ham2 * QubitOperator('Z0 Z1 Y2')
ham3

-0.1375 [] +
0.1375 [Z0 Z1] +
0.1375 [Z0 Z3] +
-0.1375 [Z1 Z3] 

In [40]:
h = Hamiltonian(QubitOperator('Z0 Z2'))
h

1.0 [Z0 Z2] 

In [41]:
sim = Simulator('projectq', trotter_circ.n_qubits)
sim

projectq simulator with 4 qubits.
Current quantum state:
[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

In [42]:
trotter_circ.params_name

['x', 'y']

In [43]:
sim.apply_circuit(trotter_circ, {'x': 1.1, 'y':2.2})
sim

projectq simulator with 4 qubits.
Current quantum state:
[-5.88501117e-01-1.53036884e-17j  4.04250005e-17-1.65618574e-15j
  1.22602527e-17+1.82461518e-16j  8.08496404e-01-2.55702286e-18j
 -1.21266342e-17-4.84940409e-16j -8.74001632e-18-3.37180978e-17j
  1.65287524e-16+1.17067449e-16j  2.55029432e-17+7.13385480e-16j
 -1.63341873e-17-5.51298044e-16j -1.05188827e-16+3.46813113e-17j
 -7.65665028e-17+2.30987632e-17j  2.24402831e-17+8.04549130e-16j
 -1.29270980e-16+1.63630239e-17j -2.48012522e-19+1.32811394e-17j
 -1.73454618e-17-4.18275182e-17j  1.80543151e-16-9.32246478e-17j]

In [44]:
print(sim.get_qs(True))

-0.5885011172553473¦0000⟩
0.808496403819592¦0011⟩


## 期望值计算

In [45]:
sim.get_expectation(h)

(-0.30733286997842074+0j)

In [46]:
h.sparse(4)

1.0 [Z0 Z2] 

In [47]:
sim.get_expectation(h)

(-0.30733286997842074+0j)

## 梯度计算


$$\left<0\right|U_l^\dagger H U_r \left|0\right>$$

In [48]:
sim = Simulator('projectq', trotter_circ.n_qubits)

In [49]:
grad_ops = sim.get_expectation_with_grad(h, trotter_circ)
grad_ops

In [50]:
f, g = grad_ops(np.array([1.1, 2.2]))
f, g

(array([[-0.30733287+0.j]]),
 array([[[5.18057842e-18+0.j, 1.90320415e+00+0.j]]]))

In [51]:
trotter_circ.no_grad()

q0: ──RX(π/2)────●───────────────────────────────────────●────RX(7π/2)────RX(π/2)────●───────────────────────────────────────●────RX(7π/2)────RX(π/2)────●───────────────────────────────────────●────RX(7π/2)────RX(π/2)────●────────────────────────────────────────●────RX(7π/2)───────H───────●───────────────────────────────────────●───────H───────────H───────●────────────────────────────────────────●───────H───────────H───────●────────────────────────────────────────●───────H───────────H───────●────────────────────────────────────────●───────H───────────H───────●─────────────────●───────H──────
                 │                                       │                           │                                       │                           │                                       │                           │                                        │                           │                                       │                           │                                        │                           │                                        │                           │                                        │                           │                 │              
q1: ─────H───────X────●─────────────────────────────●────X───────H───────────H───────X────●─────────────────────────────●────X───────H────────RX(π/2)────X────●─────────────────────────────●────X────RX(7π/2)────RX(π/2)────X────●──────────────────────────────●────X────RX(7π/2)───────H───────X────●─────────────────────────────●────X───────H───────────H───────X────●──────────────────────────────●────X───────H────────RX(π/2)────X────●──────────────────────────────●────X────RX(7π/2)────RX(π/2)────X────●──────────────────────────────●────X────RX(7π/2)────RX(π/2)────X────RZ(2.0*y)────X────RX(7π/2)──
                      │                             │                                     │                             │                                     │                             │                                     │                              │                                     │                             │                                     │                              │                                     │                              │                                     │                              │                                                                 
q2: ──RX(π/2)─────────X────●───────────────────●────X─────────RX(7π/2)───────H────────────X────●───────────────────●────X────────────H────────RX(π/2)─────────X────●───────────────────●────X─────────RX(7π/2)───────H────────────X────●────────────────────●────X────────────H────────RX(π/2)─────────X────●───────────────────●────X─────────RX(7π/2)───────H────────────X────●────────────────────●────X────────────H────────RX(π/2)─────────X────●────────────────────●────X─────────RX(7π/2)───────H────────────X────●────────────────────●────X────────────H────────────────────────────────────────────────────
                           │                   │                                               │                   │                                               │                   │                                               │                    │                                               │                   │                                               │                    │                                               │                    │                                               │                    │                                                                      
q3: ──RX(π/2)──────────────X────RZ(0.125*x)────X──────────────RX(7π/2)───────H─────────────────X────RZ(0.125*x)────X─────────────────H───────────H─────────────────X────RZ(0.125*x)────X─────────────────H────────RX(π/2)──────────────X────RZ(-0.125*x)────X──────────────RX(7π/2)───────H─────────────────X────RZ(0.125*x)────X─────────────────H────────RX(π/2)──────────────X────RZ(-0.125*x)────X──────────────RX

In [52]:
grad_ops = sim.get_expectation_with_grad(h, trotter_circ)
grad_ops

In [53]:
f, g = grad_ops(np.array([1.1, 2.2]))
f, g

(array([[-0.30733287+0.j]]), array([[[0.+0.j, 0.+0.j]]]))

In [54]:
trotter_circ.requires_grad()
for gate in trotter_circ:
    if gate.parameterized:
        if 'x' in gate.coeff:
            gate.coeff.no_grad_part('x')

In [55]:
grad_ops = sim.get_expectation_with_grad(h, trotter_circ)
grad_ops

In [56]:
f, g = grad_ops(np.array([1.1, 2.2]))
f, g

(array([[-0.30733287+0.j]]), array([[[0.        +0.j, 1.90320415+0.j]]]))

In [57]:
herm_trotter_circ = trotter_circ.hermitian()
herm_trotter_circ

q0: ──────H────────●──────────────────●───────H────────────H────────●───────────────────────────────────────●───────H────────────H────────●───────────────────────────────────────●───────H────────────H────────●───────────────────────────────────────●───────H────────────H────────●────────────────────────────────────────●───────H────────RX(-7π/2)────●───────────────────────────────────────●────RX(-π/2)────RX(-7π/2)────●────────────────────────────────────────●────RX(-π/2)────RX(-7π/2)────●────────────────────────────────────────●────RX(-π/2)────RX(-7π/2)────●────────────────────────────────────────●────RX(-π/2)──
                   │                  │                             │                                       │                             │                                       │                             │                                       │                             │                                        │                             │                                       │                             │                                        │                             │                                        │                             │                                        │              
q1: ──RX(-7π/2)────X────RZ(-2.0*y)────X────RX(-π/2)────RX(-7π/2)────X────●─────────────────────────────●────X────RX(-π/2)────RX(-7π/2)────X────●─────────────────────────────●────X────RX(-π/2)────────H────────X────●─────────────────────────────●────X───────H────────────H────────X────●──────────────────────────────●────X───────H────────RX(-7π/2)────X────●─────────────────────────────●────X────RX(-π/2)────RX(-7π/2)────X────●──────────────────────────────●────X────RX(-π/2)────────H────────X────●──────────────────────────────●────X───────H────────────H────────X────●──────────────────────────────●────X───────H──────
                                                                         │                             │                                       │                             │                                       │                             │                                       │                              │                                       │                             │                                       │                              │                                       │                              │                                       │                              │                   
q2: ──────────────────────────────────────────H──────────────────────────X────●───────────────────●────X────────────H────────RX(-7π/2)─────────X────●───────────────────●────X─────────RX(-π/2)────────H─────────────X────●───────────────────●────X────────────H────────RX(-7π/2)─────────X────●────────────────────●────X─────────RX(-π/2)────────H─────────────X────●───────────────────●────X────────────H────────RX(-7π/2)─────────X────●────────────────────●────X─────────RX(-π/2)────────H─────────────X────●────────────────────●────X────────────H────────RX(-7π/2)─────────X────●────────────────────●────X─────────RX(-π/2)──
                                                                              │                   │                                                 │                   │                                                 │                   │                                                 │                    │                                                 │                   │                                                 │                    │                                                 │                    │                                                 │                    │                        
q3: ──────────────────────────────────────────H───────────────────────────────X────RZ(0.125*x)────X─────────────────H────────RX(-7π/2)──────────────X────RZ(0.125*x)────X──────────────RX(-π/2)────RX(-7π/2)──────────────X────RZ(0.125*x)────X──────────────RX(-π/2)────────H──────────────────X───

In [58]:
grad_ops = sim.get_expectation_with_grad(h, trotter_circ, herm_trotter_circ)
grad_ops

In [59]:
f, g = grad_ops(np.array([1.1, 2.2]))
f, g

(array([[1.-1.10224407e-17j]]),
 array([[[0.00000000e+00+0.0000000e+00j, 1.38777878e-16-1.4791142e-31j]]]))

In [60]:
sim = Simulator('projectq', 1)
c1 = Circuit().ry('a', 0)
c2 = Circuit().ry('b', 0)
h = Hamiltonian(QubitOperator('Z0'))
h

1.0 [Z0] 

In [61]:
grad_ops = sim.get_expectation_with_grad(h, c1, c2)

In [62]:
f, g = grad_ops(np.array([1.1, 1.1]))
f, g

(array([[0.45359612+0.j]]), array([[[-0.44560368+0.j, -0.44560368+0.j]]]))

In [63]:
grad_ops = sim.get_expectation_with_grad(h, c1)
f, g = grad_ops(np.array([1.1]))
f ,g

(array([[0.45359612+0.j]]), array([[[-0.89120736+0.j]]]))

## 变分量子算法

In [64]:
import mindspore as ms
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")

from mindquantum.framework import MQAnsatzOnlyLayer

In [65]:
net = MQAnsatzOnlyLayer(grad_ops)

In [66]:
net()

Tensor(shape=[1], dtype=Float32, value= [ 9.99983728e-01])

In [67]:
from mindspore.nn import TrainOneStepCell
from mindspore.nn import Adam

opti = Adam(net.trainable_params(), learning_rate=0.1)
train_net = TrainOneStepCell(net, opti)
for i in range(100):
    res = train_net()
    if i % 10 == 0:
        print(f"step: {i}, res: {res}")

step: 0, res: [0.9999837]
step: 10, res: [0.59308934]
step: 20, res: [-0.42830175]
step: 30, res: [-0.9947318]
step: 40, res: [-0.93682224]
step: 50, res: [-0.9817956]
step: 60, res: [-0.997952]
step: 70, res: [-0.99615663]
step: 80, res: [-0.99998605]
step: 90, res: [-0.99947447]


## 综合案例：量子经典混合神经网络

![bloch.png](./bloch.png)

In [68]:
circ = U3('a','b','c',0)
circ

q0: ──RZ(a)────RX(-π/2)────RZ(b)────RX(π/2)────RZ(c)──

In [69]:
 psi1 = np.array([0, 0, 0])
 psi2 = np.array([0, -np.pi/2, 0])

In [70]:
print(circ.get_qs(pr=dict(zip(circ.params_name, psi1)), ket=True))
print(circ.get_qs(pr=dict(zip(circ.params_name, psi2)), ket=True))

1¦0⟩
√2/2¦0⟩
√2/2¦1⟩


In [71]:
ansatz = U3('a','b','c', 0)
ansatz

q0: ──RZ(a)────RX(-π/2)────RZ(b)────RX(π/2)────RZ(c)──

In [72]:
encoder = U3('d','e','f', 0)
encoder

q0: ──RZ(d)────RX(-π/2)────RZ(e)────RX(π/2)────RZ(f)──

In [73]:
ham = Hamiltonian(QubitOperator(''))
sim = Simulator('projectq', 1)
grad_ops = sim.get_expectation_with_grad(ham, 
                                         encoder,
                                         ansatz,
                                         encoder_params_name=encoder.params_name,
                                         ansatz_params_name=ansatz.params_name)

In [74]:
from mindquantum.framework import MQN2Layer
net = MQN2Layer(grad_ops)

In [75]:
from mindspore import Tensor
data = Tensor(np.array([psi1, psi2]).astype(np.float32))
data

Tensor(shape=[2, 3], dtype=Float32, value=
[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
 [ 0.00000000e+00, -1.57079637e+00,  0.00000000e+00]])

In [76]:
net(data)

Tensor(shape=[2, 1], dtype=Float32, value=
[[ 9.99991953e-01],
 [ 4.97162253e-01]])

In [77]:
import mindspore.nn as nn
import mindspore.ops as ops

class HybridNet(nn.Cell):
    def __init__(self, net):
        super(HybridNet, self).__init__()
        self.net = net
        self.sum = ops.ReduceSum()
    
    def construct(self, x):
        x = self.net(x)
        x = -self.sum(x, 0)
        return x

hybrid_net = HybridNet(net)
hybrid_net(data)

Tensor(shape=[1], dtype=Float32, value= [-1.49715424e+00])

In [78]:
opti = Adam(hybrid_net.trainable_params(), learning_rate = 0.1)
train_net = TrainOneStepCell(hybrid_net, opti)
for i in range(800):
    sum_fid = -train_net(data)
    if i % 50 == 0:
        print(f"step: {i}, sum of fidelity: {sum_fid}")

step: 0, sum of fidelity: [1.4971542]
step: 50, sum of fidelity: [1.7064114]
step: 100, sum of fidelity: [1.7071024]
step: 150, sum of fidelity: [1.7071068]
step: 200, sum of fidelity: [1.7071068]
step: 250, sum of fidelity: [1.7071068]
step: 300, sum of fidelity: [1.7071068]
step: 350, sum of fidelity: [1.7071067]
step: 400, sum of fidelity: [1.7071067]
step: 450, sum of fidelity: [1.7071067]
step: 500, sum of fidelity: [1.7071067]
step: 550, sum of fidelity: [1.7071067]
step: 600, sum of fidelity: [1.7071067]
step: 650, sum of fidelity: [1.7071067]
step: 700, sum of fidelity: [1.7071067]
step: 750, sum of fidelity: [1.7071067]


In [79]:
net.weight.asnumpy()

array([ 1.4284919e-02, -7.8539819e-01, -2.2062613e-16], dtype=float32)

In [80]:
print(encoder.get_qs(pr=dict(zip(encoder.params_name, net.weight.asnumpy())), ket=True))

(0.9238559626982236-0.006598716015774229j)¦0⟩
(0.38267368125677165-0.002733277752464167j)¦1⟩


In [81]:
Circuit(RY(np.pi/4).on(0)).get_qs()

array([0.92387953+0.j, 0.38268343+0.j])